In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.contrib.layers import l2_regularizer
import os


C:\Users\tkddj\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
regularizer_flag = False

In [3]:
k_regularizer = None
if regularizer_flag == True:
    k_regularizer = tf.contrib.layers.l2_regularizer(scale=8e-3)


In [4]:
def inception(inputs, o_filters, t_filters, f_filters,  pool_proj,t_reduce, f_reduce, activation=None, name=None, kernel_regularizer=None): # o:one, t:three, f:five
    one_conv=tf.layers.conv2d(inputs,filters= o_filters, kernel_size=(1,1), activation=tf.nn.relu, kernel_regularizer=kernel_regularizer,padding='same', name = name+'_one_conv')

    reduce_three=tf.layers.conv2d(inputs=inputs, filters=t_reduce, kernel_size=(3,3), kernel_regularizer=kernel_regularizer,activation=tf.nn.relu, padding='same', name= name+'_reduce_three')
    three_conv=tf.layers.conv2d(inputs=reduce_three, filters=t_filters, kernel_size=(3,3),kernel_regularizer=kernel_regularizer,activation=activation, padding='same', name = name+'_three_conv')

    reduce_five=tf.layers.conv2d(inputs=inputs, filters=f_reduce, kernel_size=(5,5), kernel_regularizer=kernel_regularizer,activation=tf.nn.relu, padding='same', name=name + '_reduce_five')
    five_conv=tf.layers.conv2d(inputs=reduce_five, filters=f_filters, kernel_size=(5,5), kernel_regularizer=kernel_regularizer, activation=activation, padding='same', name=name+'_five_conv')

    max_pool = tf.layers.max_pooling2d(inputs=inputs, pool_size=(3,3), strides=(1,1), padding='same',name=name+'_max_pool')
    max_conv = tf.layers.conv2d(inputs=max_pool, filters=pool_proj, kernel_size=(1,1), kernel_regularizer=kernel_regularizer, activation=activation, padding='same', name=name+'_max_conv')

    concat = tf.concat([one_conv, three_conv, five_conv, max_conv], 3, name=name+'_concat')

    return concat


In [5]:
from math import sqrt

def put_kernels_on_grid (kernel, pad = 1):

    '''Visualize conv. filters as an image (mostly for the 1st layer).
    Arranges filters into a grid, with some paddings between adjacent filters.
    Args:
    kernel:            tensor of shape [Y, X, NumChannels, NumKernels]
    pad:               number of black pixels around each filter (between them)
    Return:
    Tensor of shape [1, (Y+2*pad)*grid_Y, (X+2*pad)*grid_X, NumChannels].
    '''
    # get shape of the grid. NumKernels == grid_Y * grid_X
    def factorization(n):
        for i in range(int(sqrt(float(n))), 0, -1):
            if n % i == 0:
                if i == 1: print('Who would enter a prime number of filters')
                return (i, int(n / i))
    (grid_Y, grid_X) = factorization (kernel.get_shape()[3].value)
    #print ('grid: %d = (%d, %d)' % (kernel.get_shape()[3].value, grid_Y, grid_X))

    x_min = tf.reduce_min(kernel)
    x_max = tf.reduce_max(kernel)
    kernel = (kernel - x_min) / (x_max - x_min)

    # pad X and Y
    x = tf.pad(kernel, tf.constant( [[pad,pad],[pad, pad],[0,0],[0,0]] ), mode = 'CONSTANT')

    # X and Y dimensions, w.r.t. padding
    Y = kernel.get_shape()[0] + 2 * pad
    X = kernel.get_shape()[1] + 2 * pad

    channels = kernel.get_shape()[2]
    print(kernel.get_shape)
    # put NumKernels to the 1st dimension
    x = tf.transpose(x, (3, 0, 1, 2))
    # organize grid on Y axis
    x = tf.reshape(x, tf.stack([grid_X, Y * grid_Y, X, channels]))

    # switch X and Y axes
    x = tf.transpose(x, (0, 2, 1, 3))
    # organize grid on X axis
    x = tf.reshape(x, tf.stack([1, X * grid_X, Y * grid_Y, channels]))

    # back to normal order (not combining with the next step for clarity)
    x = tf.transpose(x, (2, 1, 3, 0))

    # to tf.image_summary order [batch_size, height, width, channels],
    #   where in this case batch_size == 1
    x = tf.transpose(x, (3, 0, 1, 2))

    # scaling to [0, 255] is not necessary for tensorboard
    return x



In [6]:
# placeholders and variables ...
input_x = tf.placeholder(dtype=tf.float32, shape=[None, 224, 224, 3], name='inputs')
tf.summary.image('inputs',input_x)
input_y = tf.placeholder(dtype=tf.float32, shape=[None, 120], name='input_labels')
keep_prob = tf.placeholder(dtype=tf.float32, name='keep')
is_training = tf.placeholder(dtype=tf.bool, name='is_training')
#tf.summary.image('inputs', input_x)
normalizing = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), input_x, name='input_normalizing')

# __Model__

In [7]:
#model
# stem area
low_conv1=tf.layers.conv2d(inputs= input_x, filters=64, kernel_size=(7,7), kernel_regularizer=k_regularizer,activation=tf.nn.relu, padding='same', strides=(2,2), name='stem_conv1')
low1_batch = tf.layers.batch_normalization(low_conv1, training=is_training, name='low1_bnormal')
low1_act = tf.nn.relu(low1_batch, name='low1_act')
# 112x112@64
low_max_pool1=tf.layers.max_pooling2d(inputs=low1_act, pool_size=(3,3),strides=(2,2), padding='same', name='stem_max_pool1')
# 56x56@64
low_reduce=tf.layers.conv2d(inputs=low_max_pool1, filters=64, kernel_size=(1,1), kernel_regularizer=k_regularizer,activation=tf.nn.relu, padding='same',strides=(1,1), name='stem_reduce')
low_conv2=tf.layers.conv2d(inputs=low_reduce, filters=192, kernel_size=(3,3), kernel_regularizer=k_regularizer,activation=tf.nn.relu, padding='same',strides=(1,1), name='stem_conv2')
low2_batch = tf.layers.batch_normalization(low_conv2, training=is_training, name='low2_bnormal')
low2_act = tf.nn.relu(low2_batch, name='low2_act')
# 56x56@192
low_max_pool2=tf.layers.max_pooling2d(inputs=low2_act, pool_size=(3,3), strides=(2,2), padding='same', name='stem_max_pool2')
# 28x28@192

#inception module
inception3a = inception(inputs=low_max_pool2, o_filters=64, t_filters=128, f_filters=32, pool_proj=32, t_reduce=96, f_reduce=16, activation=tf.nn.relu, name='inception3a', kernel_regularizer=k_regularizer)
batch3a = tf.layers.batch_normalization(inception3a, training=is_training, name='inception3a_bnormal')
act_batch3a = tf.nn.relu(batch3a, name='act_batch3a')
# 28x28@256
inception3b = inception(inputs=act_batch3a, o_filters=128, t_filters=192, f_filters=96, pool_proj=64, t_reduce=128, f_reduce=32, activation=None, name='inception3b', kernel_regularizer=k_regularizer)
batch3b = tf.layers.batch_normalization(inception3b, training=is_training, name='inception3b_bnormal')
act_batch3b = tf.nn.relu(batch3b, name='act_batch3b')
# 28x28@480
incept_max3 = tf.layers.max_pooling2d(inputs=act_batch3b,pool_size=(3,3), strides=(2,2), padding='same', name='inception3_max_pool')
# 14x14@336
inception4a = inception(inputs=incept_max3, o_filters=192, t_filters=208, f_filters=48, pool_proj=64, t_reduce=96, f_reduce=16, activation=tf.nn.relu,name='inception4a', kernel_regularizer=k_regularizer)
batch4a = tf.layers.batch_normalization(inception4a, training=is_training, name='inception4a_bnormal')
act_batch4a = tf.nn.relu(batch4a, name='act_batch4a')
# 14x14@512
inception4b = inception(inputs= act_batch4a, o_filters=160, t_filters=224, f_filters=64, pool_proj=64, t_reduce=112, f_reduce=24, activation=None,name='inception4b', kernel_regularizer=k_regularizer)
batch4b = tf.layers.batch_normalization(inception4b, training=is_training, name='inception4b_bnormal')
act_batch4b = tf.nn.relu(batch4b, name='act_batch4b')
# 14x14@512
inception4c = inception(inputs= act_batch4b, o_filters=128, t_filters=256, f_filters=64, pool_proj=64, t_reduce=128, f_reduce=24, activation=tf.nn.relu, name='inception4c', kernel_regularizer=k_regularizer)
batch4c = tf.layers.batch_normalization(inception4c, training=is_training, name='inception4c_bnormal')
act_batch4c = tf.nn.relu(batch4c, name='act_batch4c')
# 14x14@512
inception4d = inception(inputs=act_batch4c, o_filters=112, t_filters=288, f_filters=64, pool_proj=128, t_reduce=144, f_reduce=32, activation=None, name='inception4d', kernel_regularizer=k_regularizer)
batch4d = tf.layers.batch_normalization(inception4d, training=is_training, name='inception4d_bnormal')
act_batch4d = tf.nn.relu(batch4d, name='act_batch4d')
# 14x14@528
inception4e = inception(inputs=act_batch4d, o_filters=256, t_filters=320, f_filters=128, pool_proj=128, t_reduce=160, f_reduce=32, activation=tf.nn.relu,name='inception4e', kernel_regularizer=k_regularizer)
batch4e = tf.layers.batch_normalization(inception4e, training=is_training, name='inception4e_bnormal')
act_batch4e = tf.nn.relu(batch4e, name='act_batch4e')
# 14x14@832
incept_max4 = tf.layers.max_pooling2d(inputs=act_batch4e, pool_size=(3,3), strides=(2,2), padding='same', name='inception4_max_pool')
# 7x7@ 618
inception5a = inception(inputs=incept_max4, o_filters=256, t_filters=320, f_filters=128, pool_proj=128, t_reduce=160, f_reduce=32, activation=tf.nn.relu,name='inception5a', kernel_regularizer=k_regularizer)
batch5a = tf.layers.batch_normalization(inception5a, training=is_training, name='inception5a_bnormal')
act_batch5a = tf.nn.relu(batch5a, name='act_batch5a')
# 7x7@ 618
inception5b = inception(inputs=act_batch5a, o_filters=384, t_filters=384, f_filters=128, pool_proj=128, t_reduce=192, f_reduce=48, activation=tf.nn.relu, name='inception5b', kernel_regularizer=k_regularizer)
batch5b = tf.layers.batch_normalization(inception5b, training=is_training, name='inception5b_bnormal')
act_batch5b = tf.nn.relu(batch5b, name='act_batch5b')
# 7x7@ 863
avg_pool = tf.layers.average_pooling2d(inputs=act_batch5b, pool_size=(7,7), strides=(1,1), padding='valid',name='avg_pool')



## __Take summaries__

In [8]:
tf_var= tf.global_variables()
tf_var = [var for var in tf_var if var.name.split('/')[1][:-2] == 'kernel' or var.name.split('/')[1][:-2] == 'bias']


In [9]:
for var in tf_var:
    tf.summary.histogram(var.name[:-2], var)

In [10]:
# FC area
drop = tf.layers.dropout(avg_pool, keep_prob)
flat = tf.layers.flatten(drop)
fc1 = tf.layers.dense(inputs=flat, units= 600, activation=tf.nn.relu)
#fc2 = tf.layers.dense(inputs=fc1, units=256, activation=tf.nn.relu)
fc2 = tf.layers.dense(inputs=fc1, units=120, activation=tf.nn.relu)


In [11]:
#cost and prediction
pred = tf.nn.softmax(fc2, name='prediction')
reg_loss =tf.losses.get_regularization_loss()
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc2, labels=input_y), name='cost') + reg_loss
tf.summary.scalar('train_loss', cost)
correct = tf.cast(tf.equal(tf.argmax(pred, 1), tf.argmax(input_y, 1)), tf.float32)
accuracy = tf.reduce_mean(correct, name='accuarcy')
tf.summary.scalar('accuracy', accuracy)
#training
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)  # batch normalization
with tf.control_dependencies(update_ops):
    opt = tf.train.AdamOptimizer(3e-5).minimize(cost)



In [12]:
with tf.variable_scope('stem_conv1'):
    tf.get_variable_scope().reuse_variables()
    weights= tf.get_variable('kernel')
    print(weights)
    grid = put_kernels_on_grid(weights)
    imgsum=tf.summary.image('stem_conv1/kernels', grid, max_outputs=1)

<tf.Variable 'stem_conv1/kernel:0' shape=(7, 7, 3, 64) dtype=float32_ref>
<bound method Tensor.get_shape of <tf.Tensor 'stem_conv1_1/truediv:0' shape=(7, 7, 3, 64) dtype=float32>>


In [13]:
sess = tf.Session()
merged_summary = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('D:/JupyterDir/TensorBoard/Inception/train/3e-5_batch50_bnormal_cvnormal', sess.graph)
test_writer = tf.summary.FileWriter('D:/JupyterDir/TensorBoard/Inception/test/3e-5_batch50_bnormal_cvnormal')
sess.run(tf.global_variables_initializer())

# __Reading Files__

In [14]:
# read names
os.chdir('D:/JupyterDir/aug_Resized')
folder_list = os.listdir()

data_dict = dict()
one_hot_labels = dict()
i = 0
for folder in folder_list:
    os.chdir(folder)
    file_list = os.listdir()
    label = np.zeros([120])
    label[i] = 1
    one_hot_labels[folder] = label
    for file in file_list:
        data_dict[folder + '/' + file] = folder
    os.chdir('../')
    i+=1

In [15]:
## shuffling
import random
name_x = list(data_dict.keys())
random.shuffle(name_x)

labels = list()
for name in name_x:
    labels.append(one_hot_labels[data_dict[name]])


In [16]:
# setting
train_name = name_x[0:int(len(name_x) * 0.6)]
cv_name = name_x[int(len(name_x) *0.6):int(len(name_x) * 0.8)]
test_name = name_x[int(len(name_x) *0.8):int(len(name_x))]
train_labels = labels[0:int(len(name_x)*0.6)]
cv_labels = labels[int(len(name_x) *0.6):int(len(name_x) * 0.8)]
test_labels = labels[int(len(name_x) *0.8):int(len(name_x))]
batch_size = 50
batch_x = [0] * batch_size
batch_y= [0] * batch_size
all_pre = np.zeros([0])
batch_cv = [0] * int(len(cv_name) / 10)

cv_check = dict()
cv_csv = dict()
for i in range(len(cv_name)):
    cv_check[cv_name[i].split('/')[0]] = {'right':0, 'num_data':0}
    cv_csv[cv_name[i].split('/')[0]] = list()
for i in range(len(cv_name)):
    cv_check[cv_name[i].split('/')[0]]['num_data'] += 1
    
cv_batch_size = int(len(cv_name)/40)
cv_data = [0] * cv_batch_size

print('checkpoint 2')
for i in range(cv_batch_size):
    cv_data[i] = cv2.imread(cv_name[i])

train_set = list(zip(train_name, train_labels))
all_acc = list()

checkpoint 2


# __Learning Process__

In [ ]:
os.chdir('D:/JupyterDir/aug_Resized')

#sess = tf.Session()
#sess.run(tf.global_variables_initializer())

print('checkpoint 4')
cnt = 0
for epoch in range(101):
    print('epoch{}'.format(epoch))
    bnd = 0
    random.shuffle(train_set)
    for j in range(int(len(train_name) / batch_size)):
        #print('batch{}'.format(j))
        for i in range(batch_size):
            if (bnd >= len(train_name)):
                break
            batch_x[i] = cv2.imread(train_set[bnd][0])
            batch_y[i] = train_set[bnd][1]
            bnd += 1
        normed=sess.run(normalizing, feed_dict={input_x: batch_x[0:i]})
        _, loss = sess.run([opt, cost],feed_dict={input_x: normed, input_y: batch_y[0:i], keep_prob: 0.4, is_training: True})
        if j % 10 == 0:
            summary = sess.run(merged_summary,feed_dict={input_x: normed, input_y: batch_y[0:i], keep_prob: 1, is_training: False})
        #train_loss.append(loss)
            train_writer.add_summary(summary, epoch*batch_size + j)
    print('epoch{}:loss{}'.format(epoch, loss))
    
    # accuracy check and save
    if epoch % 5 == 0:
        bnd = 0
        for j in range(int(len(cv_name) / cv_batch_size) + 1):
            for i in range(cv_batch_size):
                if bnd >= len(cv_name):
                    break
                cv_data[i] = cv2.imread(cv_name[bnd])
                batch_cv[i] = cv_labels[bnd]
                bnd += 1
            normed = sess.run(normalizing, feed_dict={input_x:cv_data[0:i]})
            summary, pre = sess.run([merged_summary,correct], feed_dict={input_x: normed, input_y: batch_cv[0:i], keep_prob: 1, is_training: False})
            all_pre = np.concatenate([all_pre, pre])
            test_writer.add_summary(summary, cnt)
            cnt+=1
        print('{}th: cv acc:{}'.format(epoch, np.mean(all_pre)))
        for k in range(len(all_pre)):
            if all_pre[k] > 0:
                cv_check[cv_name[k].split('/')[0]]['right'] += 1
        for name in cv_check.keys():
            each_acc = cv_check[name]['right'] / cv_check[name]['num_data']
            cv_csv[name].append(each_acc)
            print('{}, {:.5f}'.format(name, each_acc))
        for k in range(len(all_pre)):
            cv_check[cv_name[k].split('/')[0]]['right'] = 0
        all_acc.append(all_pre)
        all_pre = np.zeros([0])
        
        #acc = sess.run(accuracy, feed_dict={input_x: cv_data, input_y: cv_labels, keep_prob: 1, is_training: False})


checkpoint 4
epoch0
epoch0:loss4.272224426269531
0th: cv acc:0.05382367267413541
Kerry_blue_terrier, 0.06475
bloodhound, 0.06897
toy_poodle, 0.06604
curly, 0.04464
Great_Dane, 0.05310
Dandie_Dinmont, 0.05000
Saluki, 0.04878
wire, 0.02362
Welsh_springer_spaniel, 0.01550
Airedale, 0.05917
bluetick, 0.04918
kelpie, 0.04065
flat, 0.04839
Sealyham_terrier, 0.08876
affenpinscher, 0.04651
collie, 0.04000
standard_poodle, 0.04878
komondor, 0.07627
Rottweiler, 0.08036
borzoi, 0.04615
golden_retriever, 0.01905
Appenzeller, 0.02419
dingo, 0.04800
Newfoundland, 0.05806
malamute, 0.04930
miniature_poodle, 0.04132
Bernese_mountain_dog, 0.04678
Irish_terrier, 0.08889
Rhodesian_ridgeback, 0.07801
soft, 0.07273
Mexican_hairless, 0.08209
Maltese_dog, 0.06944
keeshond, 0.04762
kuvasz, 0.03540
groenendael, 0.03448
Samoyed, 0.07910
Great_Pyrenees, 0.05233
Chesapeake_Bay_retriever, 0.06061
Ibizan_hound, 0.05634
basenji, 0.09396
Tibetan_terrier, 0.01840
giant_schnauzer, 0.02479
cairn, 0.00000
English_setter,

# __Save File__

In [26]:
os.chdir('D:/JupyterDir')
l = np.asarray(all_acc)

In [37]:
cv_acc = np.mean(l,1)
np.savetxt('cv_acc.csv',cv_acc * 100,delimiter=',')

In [41]:
import csv
with open('each_acc.csv', 'w') as file:
    writer=csv.writer(file,lineterminator='\n')
    for name in cv_csv.keys():
        file.write(name + ',')
        writer.writerow(cv_csv[name])

# __Test Accuracy__

In [45]:
os.chdir('D:/JupyterDir/aug_Resized')
bnd = 0
test_data = [0]*cv_batch_size
batch_test = [0]*cv_batch_size
t_pre_list = list()
for j in range(int(len(test_name) / cv_batch_size) + 1):
    for i in range(cv_batch_size):
        if bnd >= len(test_name):
            break
        test_data[i] = cv2.imread(test_name[bnd])
        batch_test[i] = test_labels[bnd]
        bnd += 1
    normed = sess.run(normalizing, feed_dict={input_x:test_data[0:i]})
    test_pre = sess.run(correct, feed_dict={input_x: normed, input_y: batch_test[0:i], keep_prob: 1, is_training: False})
    all_pre = np.concatenate([all_pre, pre])
    t_pre_list.append(test_pre) 
    #test_writer.add_summary(summary, cnt)
    cnt+=1


In [74]:
tt_ = np.zeros([0])
for pre in t_pre_list:
    tt_ = np.concatenate([tt_,pre])
print('test accuracy:{:.2f}'.format(np.mean(tt_)))

test accuracy:0.55


In [76]:
#Checking Real Data
# read names
os.chdir('D:/JupyterDir/Resized')
folder_list = os.listdir()

data_dict = dict()
one_hot_labels = dict()
i = 0
for folder in folder_list:
    os.chdir(folder)
    file_list = os.listdir()
    label = np.zeros([120])
    label[i] = 1
    one_hot_labels[folder] = label
    for file in file_list:
        data_dict[folder + '/' + file] = folder
    os.chdir('../')
    i+=1
    
name_x = list(data_dict.keys())
random.shuffle(name_x)

labels = list()
for name in name_x:
    labels.append(one_hot_labels[data_dict[name]])

In [77]:
os.chdir('D:/JupyterDir/Resized')
bnd = 0
test_data = [0]*cv_batch_size
batch_test = [0]*cv_batch_size
t_pre_list = list()
for j in range(int(len(name_x) / cv_batch_size) + 1):
    for i in range(cv_batch_size):
        if bnd >= len(name_x):
            break
        test_data[i] = cv2.imread(name_x[bnd])
        batch_test[i] = labels[bnd]
        bnd += 1
    normed = sess.run(normalizing, feed_dict={input_x:test_data[0:i]})
    test_pre = sess.run(correct, feed_dict={input_x: normed, input_y: batch_test[0:i], keep_prob: 1, is_training: False})
    all_pre = np.concatenate([all_pre, pre])
    t_pre_list.append(test_pre)
    #test_writer.add_summary(summary, cnt)
    cnt+=1

In [ ]:
tt_ = np.zeros([0])
for pre in t_pre_list:
    tt_ = np.concatenate([tt_,pre])

In [79]:
print('test accuracy:{:.4f}'.format(np.mean(tt_)))

test accuracy:0.7119


In [ ]:
train_check = dict()

for i in range(len(train_name)):
    train_check[train_name[i].split('/')[0]] = 0

In [ ]:
for i in range(len(train_name)):
    train_check[train_name[i].split('/')[0]] += 1
    
for i in list(train_check.keys()):
    print('{}:{}'.format(i, train_check[i]))

In [35]:
print(len(cv_name), k)
print(int(len(cv_name) % cv_batch_size))
all_pre.shape

16464 16488
24


(0,)

In [ ]:
#sanity check
with tf.Session() as test_sess:
    sc_loss = list()
    test_sess.run(tf.global_variables_initializer())
    for epoch in range(1000):
        for i in range(20):
            batch_x[i] = cv2.imread(train_set[i][0])
            batch_y[i] = train_set[i][1]
        normed=test_sess.run(normalizing, feed_dict={input_x: batch_x[0:cnt]})
        _, loss = test_sess.run([opt, cost],feed_dict={input_x: normed, input_y: batch_y[0:cnt], keep_prob: 1, is_training: True})
        sc_loss.append(loss)
        print('epoch {}'.format(epoch))
    pre = test_sess.run(accuracy, feed_dict={input_x: batch_x[0:20], input_y: batch_y[0:20], keep_prob: 1, is_training: False})
    print(pre)
    